<a href="https://colab.research.google.com/github/neemiasbsilva/MLLMs-Teoria-e-Pratica/blob/main/use-cases/agent_rag_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-On: Construindo um Agent RAG com LangGraph

**Objetivo**: Criar um agente de Retrieval-Augmented Generation (RAG) do zero usando `langgraph`. Este agente será capaz de responder perguntas com base em um documento da web, e o mais importante, ele saberá "decidir" o que fazer se não encontrar a informação.

Importante: Para rodar este Colab, você precisa de uma API Key do Google AI Studio.

 - Acesse [aistudio.google.com/app/apikey](aistudio.google.com/app/apikey)

 - Clique em "Create API key" e copie a chave.

 - No Colab, clique no ícone de "Chave" na barra lateral esquerda, crie um novo "secret" chamado GOOGLE_API_KEY e cole sua chave lá.

## Instalação e Configuração

Primeiro, vamos instalar todas as bibliotecas necessárias e configurar nossa chave de API do Google Gemini.

In [ ]:
!pip install -qU langgraph langchain langchain_google_genai langchain_community langchain_huggingface faiss-cpu sentence-transformers beautifulsoup4 -q

In [ ]:
import os
import getpass
from google.colab import userdata

# Tenta carregar a chave do Colab Secrets
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
except:
    # Se não funcionar, pede para o usuário digitar
    print("Chave da API não encontrada no Colab Secrets.")
    print("Cole sua GOOGLE_API_KEY abaixo e pressione Enter:")
    os.environ['GOOGLE_API_KEY'] = getpass.getpass()

if not os.environ.get('GOOGLE_API_KEY'):
    raise Exception("API Key do Google não configurada. O notebook não pode continuar.")

print("API Key configurada com sucesso!")

## Preparando os Dados (o "R" do RAG)

Nosso agente precisa de uma base de conhecimento. Vamos usar uma página web como nossa "memória externa".

1. Carregar: Baixar o conteúdo de uma página web.
2. Dividir: Quebrar o texto em pedaços (chunks).
3. Embarcar: Transformar cada pedaço em vetores (embeddings).
4. Armazenar: Salvar tudo em um banco de dados vetorial (FAISS) que funciona em memória.

In [50]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


print("Carregando o documento da web...")
# 1. Carregar: Vamos usar um artigo sobre o LangChain
loader = WebBaseLoader(
    "https://python.langchain.com/docs/expression_language/"
)
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Criando o Vector Store (FAISS)...")
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever()

print("\n--- Teste Rápido do Retriever ---")
test_docs = retriever.invoke("O que é LCEL?")
print(f"Encontrados {len(test_docs)} documentos relevantes para 'LCEL'.")
print("Retriever pronto!")

Carregando o documento da web...
Criando o Vector Store (FAISS)...

--- Teste Rápido do Retriever ---
Encontrados 4 documentos relevantes para 'LCEL'.
Retriever pronto!


## Definindo o Estado do Grafo (A "Memória")


In [51]:
from typing import TypedDict, List
from langchain_core.documents import Document

class GraphState(TypedDict, total=False):
    """
    Representa o estado do nosso agente.
    """
    question: str       # A pergunta original do usuário
    documents: List[Document] # Documentos recuperados pelo RAG
    context: str        # O contexto formatado para o LLM
    generation: str     # A resposta final do LLM
    error: str          # Mensagem de erro, se houver

## Definindo os Nós (Nodes)

In [53]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. O Nó de Recuperação
def retrieve_node(state: GraphState) -> GraphState:
    """
    Busca documentos no vector store com base na pergunta.
    """
    print("--- 1. EXECUTANDO NODE: retrieve_node ---")
    question = state["question"]
    documents = retriever.invoke(question)
    print(f"Documentos recuperados: {len(documents)}")

    return {"documents": documents}

def augment_prompt_node(state: GraphState) -> GraphState:
    """
    Prepara o contexto a partir dos documentos recuperados.
    """
    print("--- 2. EXECUTANDO NODE: augment_prompt_node ---")

    # Junta o conteúdo dos documentos
    context = "\n\n---\n\n".join(
        [doc.page_content for doc in state["documents"]]
    )
    # print(f"Context: {context}")
    return {"context": context}

# 3. O  Nó de Geração de Resposta
def generate_node(state: GraphState) -> GraphState:
    """
    Gera uma resposta usando o LLM (com o contexto).
    """
    print("--- 3. EXECUTANDO NODE: generate_node (RAG) ---")

    # Template do prompt RAG
    template = """Você é um assistente de IA. Use o contexto fornecido
    para responder à pergunta do usuário da melhor forma possível.

    Contexto:
    {context}

    Pergunta:
    {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Modelo LLM (Gemini)
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    # llm = ChatGoogleGenerativeAI(model="gemini-pro")

    # Cria a "chain" (Prompt + LLM + Parser)
    chain = prompt | llm | StrOutputParser()

    # Invoca a chain
    generation = chain.invoke({
        "context": state["context"],
        "question": state["question"]
    })

    return {"generation": generation}

# 4. O Nó de Fallback/Erro
def error_node(state: GraphState) -> GraphState:
    """
    Gera uma resposta padrão quando nenhum documento é encontrado.
    """
    print("--- 3b. EXECUTANDO NODE: error_node (Fallback) ---")

    error_message = "Desculpe, não consegui encontrar nenhuma informação sobre isso em minha base de dados."

    return {"generation": error_message}

## Definindo as "Decisões" (Arestas Condicionais)

In [54]:
def should_augment(state: GraphState) -> str:
    """
    Decide para qual node ir após a recuperação.
    """
    print("--- DECISÃO: should_augment ---")

    if state["documents"]:
        print("Decisão: Documentos encontrados. Seguindo para 'augment_prompt'")
        return "augment"
    else:
        print("Decisão: Nenhum documento encontrado. Seguindo para 'error'")
        return "fallback"

## Montando o Grafo

1. Instanciar o `StateGraph`.
2. Adicionar os Nós.
3. Definir o ponto de partida (`set_entry_point`).
4. Adicionar as conexões (`add_edge`) e as decisões (`add_conditional_edge`).
5. Compilar o grafo.

In [55]:
from langgraph.graph import StateGraph, END

print("Montando o grafo...")

workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve_node)
workflow.add_node("augment_prompt", augment_prompt_node)
workflow.add_node("generate", generate_node)
workflow.add_node("error", error_node)


workflow.set_entry_point("retrieve")

# A aresta condicional
workflow.add_conditional_edges(
    # O nó de onde a decisão sai (como você apontou)
    source="retrieve",

    # A função que decide o caminho (retorna "augment" ou "fallback")
    path=should_augment,

    # O mapa de destinos
    path_map={
        "augment": "augment_prompt",
        "fallback": "error"
    }
)


workflow.add_edge("augment_prompt", "generate")

workflow.add_edge("generate", END)
workflow.add_edge("error", END)


# Compilar o grafo!
app = workflow.compile()
print("Grafo compilado com sucesso!")

Montando o grafo...
Grafo compilado com sucesso!


## Executando o Agente RAG

In [56]:
print("--- INICIANDO TESTE 1 (CAMINHO FELIZ) ---")
question = "O que é LangChain?"

final_state = app.invoke({"question": question})

print("\n--- RESPOSTA FINAL (Teste 1) ---")
print(final_state["generation"])

--- INICIANDO TESTE 1 (CAMINHO FELIZ) ---
--- 1. EXECUTANDO NODE: retrieve_node ---
Documentos recuperados: 4
--- DECISÃO: should_augment ---
Decisão: Documentos encontrados. Seguindo para 'augment_prompt'
--- 2. EXECUTANDO NODE: augment_prompt_node ---
--- 3. EXECUTANDO NODE: generate_node (RAG) ---

--- RESPOSTA FINAL (Teste 1) ---
Com base no contexto fornecido, LangChain é:

Uma ferramenta que oferece a maneira mais fácil de começar a construir **agentes e aplicações alimentadas por LLMs** (Large Language Models).

**Pontos-chave sobre LangChain:**

*   Permite conectar-se a provedores de LLMs como OpenAI, Anthropic e Google com menos de 10 linhas de código.
*   Fornece uma **arquitetura de agente pré-construída** e integrações de modelo para incorporar LLMs de forma rápida e contínua.
*   É recomendado para quem deseja construir **agentes e aplicações autônomas rapidamente**.
*   Oferece uma **interface de modelo padrão** que padroniza a interação com diferentes provedores, permit

In [57]:
print("\n\n--- INICIANDO TESTE 2 (CAMINHO DE FALLBACK) ---")
question_fallback = "Qual é a capital da Eslovênia?"


final_state_fallback = app.invoke({"question": question_fallback})

print("\n--- RESPOSTA FINAL (Teste 2) ---")
print(final_state_fallback["generation"])



--- INICIANDO TESTE 2 (CAMINHO DE FALLBACK) ---
--- 1. EXECUTANDO NODE: retrieve_node ---
Documentos recuperados: 4
--- DECISÃO: should_augment ---
Decisão: Documentos encontrados. Seguindo para 'augment_prompt'
--- 2. EXECUTANDO NODE: augment_prompt_node ---
--- 3. EXECUTANDO NODE: generate_node (RAG) ---

--- RESPOSTA FINAL (Teste 2) ---
O contexto fornecido é sobre a criação de um agente de IA usando LangChain e não contém informações sobre a capital da Eslovênia.
